In [1]:
# %load LDA-Copy1.py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
import ast
import util
import pickle
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
import gensim
import pyLDAvis.gensim

In [2]:
df = util.get_processed_data("./data/collections_math.csv", True)
# df =pd.read_csv("./Final_Data/collections_all_science_out-temp_lda.csv")

In [3]:
df.head()

,collection_id,sequence_id,resource_id,title,description,text
0,0008d66a-753f-4639-8634-81bb3abb3269,3,2ee6f80f-0851-4cfa-b4bf-2655e9c46ab7,Solve the Linear equation: _______,Solve the Linear equation: [2],Solve Linear equation Solve Linear equation
1,0008d66a-753f-4639-8634-81bb3abb3269,4,81c6995c-dd95-418e-a8c4-c22d8ccd32e9,Solve the linear equation: _______,Solve the linear equation: [-18],Solve linear equation Solve linear equation
2,0008d66a-753f-4639-8634-81bb3abb3269,1,231eb4ad-d0e8-4e94-a552-f8bd2358a47a,Solve the linear equation: _______,Solve the linear equation: [1/2] ...,Solve linear equation Solve linear equation Pl...
3,0008d66a-753f-4639-8634-81bb3abb3269,2,0b248202-12a9-405b-acd1-4ab8250e4198,"If , then _______","If , then [1/5]. &nbsp;Please writ...",nbsp Please write answer fraction
4,001bf2c6-8ede-478a-9b8b-d7750488cb1b,1,1b922fae-619f-4f52-9551-663b3206e4e5,Lesson 11,I'll say an addition or subtraction sentence. ...,Lesson say addition subtraction sentence say a...


In [4]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

In [7]:
df.text.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)
df['clean'] = df.text.apply(clean)

df.clean = df.clean.apply(lambda x: x.split())
dictionary = gensim.corpora.Dictionary(df.clean)
df['clean_matrix'] = df.clean.apply(lambda x: dictionary.doc2bow(x))

file_Name = './PickelFiles/lda_dictionary.model'
fileObject = open(file_Name,'wb') 

pickle.dump(dictionary,fileObject)   

fileObject.close()
# df.Summarisation.replace({r'[^\x00-\x7F]+':''}, regex=True, inplace=True)
# df['clean'] = df.Summarisation.apply(clean)

# df.clean = df.clean.apply(lambda x: x.split())
# dictionary = gensim.corpora.Dictionary(df.clean)
# df['clean_matrix'] = df.clean.apply(lambda x: dictionary.doc2bow(x))

In [8]:
df.clean

0        [solve, linear, equation, solve, linear, equat...
1        [solve, linear, equation, solve, linear, equat...
2        [solve, linear, equation, solve, linear, equat...
3                  [nbsp, please, write, answer, fraction]
4        [lesson, say, addition, subtraction, sentence,...
5        [application, problem, talk, partner, second, ...
6        [adding, subtracting, fraction, like, unit, fl...
7        [lesson, concept, development, tell, partner, ...
8        [lesson, student, debrief, subtract, fraction,...
9        [subtracting, fraction, whole, number, say, su...
10       [cuisenaire, environment, click, rod, choose, ...
11       [application, problem, application, problem, r...
12       [break, apart, fraction, fluency, activity, re...
13       [lesson, concept, development, draw, tape, dia...
14       [lesson, decompose, fraction, sum, smaller, un...
15       [lesson, student, debrief, decompose, fraction...
16       [count, equivalent, fraction, fluency, activit.

In [8]:
lda = gensim.models.ldamodel.LdaModel
np.random.seed(1)
ldamodel = lda(
    df.clean_matrix, 
    num_topics=20, 
    id2word = dictionary, 
    passes=100,
    minimum_probability=0.0
)

In [9]:
for idx, topic in ldamodel.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.036*"data" + 0.029*"sequence" + 0.027*"mean" + 0.026*"value" + 0.023*"plot" + 0.020*"would" + 0.019*"describe" + 0.017*"set" + 0.016*"sample" + 0.015*"formula"
Topic: 1 
Words: 0.038*"box" + 0.030*"relationship" + 0.030*"group" + 0.023*"type" + 0.019*"paper" + 0.018*"many" + 0.017*"test" + 0.014*"need" + 0.014*"piece" + 0.013*"geometric"
Topic: 2 
Words: 0.085*"one" + 0.075*"round" + 0.072*"nearest" + 0.072*"inch" + 0.033*"real" + 0.030*"foot" + 0.023*"distance" + 0.022*"range" + 0.022*"radical" + 0.020*"vertical"
Topic: 3 
Words: 0.111*"number" + 0.089*"line" + 0.077*"find" + 0.040*"form" + 0.034*"write" + 0.030*"slope" + 0.030*"product" + 0.030*"point" + 0.028*"cm" + 0.027*"two"
Topic: 4 
Words: 0.094*"nbsp" + 0.081*"b" + 0.061*"c" + 0.058*"following" + 0.054*"f" + 0.046*"true" + 0.039*"e" + 0.036*"sqrt" + 0.035*"statement" + 0.035*"g"
Topic: 5 
Words: 0.078*"solve" + 0.058*"using" + 0.058*"problem" + 0.034*"word" + 0.033*"division" + 0.032*"model" + 0.029*"diagram

In [10]:
# df['lda_topic_index'] = df.clean_matrix.apply(lambda x: sorted(ldamodel[x], key=lambda tup: -1*tup[1])[0][0])

# df['lda_topic'] = df.lda_topic_index.apply(lambda x: ldamodel.print_topic(x, 10))

# for index, score in sorted(ldamodel[df.clean_matrix[1]], key=lambda tup: -1*tup[1]):
#     print("\nScore: {}\t \nTopic: {}".format(score, ldamodel.print_topic(index, 20)))



In [11]:
vis_data = pyLDAvis.gensim.prepare(ldamodel,df.clean_matrix,dictionary)

pyLDAvis.display(vis_data)

/home/bukka/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [12]:
ldamodel.save('./PickelFiles/lda_25_sc.model')
ldamodel = ldamodel.load('./PickelFiles/lda_25_sc.model')

In [13]:
# doc_topics = ldamodel.get_document_topics(df.clean_matrix[1],per_word_topics=True)[0]

df['lda_topics'] = df.clean_matrix.apply(lambda x: ldamodel.get_document_topics(x,per_word_topics=True)[0])
# df.to_csv('./data/collections_all_science_out-temp.csv')

In [20]:
df.lda_topics[0]

[(0, 0.0071428576),
 (1, 0.0071428576),
 (2, 0.0071428576),
 (3, 0.0071428576),
 (4, 0.0071428576),
 (5, 0.0071428576),
 (6, 0.0071428576),
 (7, 0.0071428576),
 (8, 0.0071428576),
 (9, 0.0071428576),
 (10, 0.0071428576),
 (11, 0.0071428576),
 (12, 0.0071428576),
 (13, 0.0071428576),
 (14, 0.0071428576),
 (15, 0.0071428576),
 (16, 0.0071428576),
 (17, 0.0071428576),
 (18, 0.8642857),
 (19, 0.0071428576)]

In [16]:
df.to_csv("./data/collections_all_maths_lda.csv",index=False)

In [17]:
df.shape

(27743, 9)

In [25]:
dictionary.load('./PickelFiles/lda_dictionary.model')
